In [53]:
function createPerson(first_name, last_name){
    return {
        first_name,
        last_name: last_name,
        ["12" + "3"]: 123
    }
}
var john = createPerson("John", "Dow")
var Endy = createPerson("Endy", "Warhol")
var rand_key = Math.random() > .5 ? "first_name" : "last_name";
john[rand_key];

'John'

В объектах мы не только выражаем с вами какие-то структуры, но и используем их как хранилище пар ключ/значение:

> Должен заметить что у нас теперь есть специализированный класс для этого (`Map`)

In [52]:
var str = "Very very long string...";

var statistic = {}
for (let char of str){
    // если нет ключика char в объекте statistic то добавим его
    if (!(char in statistic)){
        statistic[char] = 0
    }
    statistic[char]++
}
console.log(statistic);

{
  V: 1,
  e: 2,
  r: 3,
  y: 2,
  ' ': 3,
  v: 1,
  l: 1,
  o: 1,
  n: 2,
  g: 2,
  s: 1,
  t: 1,
  i: 1,
  '.': 3
}


Давайте напиишм функцию которая будет создавать объектики кошки:

In [3]:
function createCat(nickname){
    return {
        nickname: nickname,
        age: Math.round(12 * Math.random),
        sayHello: function(){
            console.log(`Hello! My name is ${this.nickname}`)
        }
    }
}
var barsik = createCat("Barsik")
var luparik = createCat("Luparik")
barsik.sayHello()
luparik.sayHello()
console.log("Is the same object:", Object.is(barsik.sayHello, luparik.sayHello))

Hello! My name is Barsik
Hello! My name is Luparik
Is the same object: false


Думаю, что вы видите некоторую странность, у каждого объекта сейчас будет собственная функция `sayHello`, но есть ли более удобный способ делать подобные проекты?

In [6]:
var additional_prototype = {
    sayName: function(){
        console.log(this.nickname)
    }
}

var cat_prototype = {
    sayHello: function(){
        console.log(`Hello! My name is ${this.nickname}`)
    },
    __proto__: additional_prototype
}

function createCat(nickname){
    return {
        nickname: nickname,
        age: Math.round(12 * Math.random),
       __proto__: cat_prototype
    }
}

var barsik = createCat("Barsik")
var luparik = createCat("Luparik")
barsik.sayHello()
luparik.sayHello()
luparik.sayName()

console.log("Is the same object:", Object.is(barsik.sayHello, luparik.sayHello))

Hello! My name is Barsik
Hello! My name is Luparik
Luparik
Is the same object: true


Теперь у нас общая функция на все экземпляры кошек. Мы даже можем поменять функцию в прототипе на новую и поведение метода у всех кошек изменится:

In [7]:
cat_prototype.sayHello = function(){
    console.log(`[NEW] Hello! My name is ${this.nickname}`)
}

barsik.sayHello()
luparik.sayHello()

[NEW] Hello! My name is Barsik
[NEW] Hello! My name is Luparik


Вуаля!

Для того чтобы удобнее создать объект из прототипа, а точнее объект у которого аттрибут `__proto__` будет ссылать на какой-то другой объект можно воспользоваться функцией `Object.create`.

Эта функция получает 1 аргумент:
- `prototype`
     Объект который будет прототипом нового объекта
и возвращает новый объект у которого свойство `__proto__` ссылается на значение аргумента `prototype`

In [3]:
var person_prototype = {
  isHuman: false,
  printIntroduction: function() {
    console.log(`My name is ${this.name}. Am I human? ${this.isHuman}`);
  }
};

var me = Object.create(person_prototype);
me.name = 'Matthew'; // "name" is a property set on "me", but not on "person"
me.isHuman = true; // inherited properties can be overwritten

me.printIntroduction();

My name is Matthew. Am I human? true


# Функции конструкторы

**Функция конструктор** - функция которая конструирует новые объекты.
> Принято называть с большой буквы

Функция конструктор вызывается через оператор `new`. если вызвана она верно, то контекст выполнения `this` будет равен новому пустому объекту. Функция должна заполнить этот объект аттрибутами.

In [10]:
function Cat(nickname, age){
    this.nickname = nickname
    this.age = age
}
Cat.prototype.sayHello = function(){
    console.log(`Hello! My name is ${this.nickname}`)
}
var luparik = new Cat("Luparik", 12)
var barsik = new Cat("Barsik", 1)
console.log(luparik, barsik)

Cat { sayHello: [Function] }
Cat { nickname: 'Luparik', age: 12 } Cat { nickname: 'Barsik', age: 1 }
